In [171]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#  PATTERN RECOGNITION

## Assignment 5


#### 1. Derive the restricted Boltzmann machine algorithm that you will implement, and explain your derivation. Implement the training and inference algorithms for RBM. Train RBMs with 20, 100 and 500 hidden nodes to generate MNIST images using the training data set. Generate MNIST images from the ones in the testing data set that have 20%, 50% and 80% pixels missing/removed. You are free to choose whether you want to use binary nodes or floating point nodes, but the derivation has to match the implementation.

### - Derivation

The goal of boltzmann machine is to model a set of observed data in terms of a set of visible random variables $v$ and a set of latent random variables $h$. Due to the relationship between boltzmann machines nad neural netowrks, the random variables are known as units. The role of these visible units is to approximate the true distribution of the data and the role of latent variables, is that they extend the expressiveness of the model by capturing underlying features in the observed data. The latent variables are the hidden units, as we cannot get them directly from the observed data and are marginalized over to obtain the likelihood of the observed data

$ p(v;\theta) = \sum_h{p(v,h;\theta)}$

where $p(v,h;\theta) $ is the jojnt probability distribution over the visible and hidden units based on the current model parameters $\theta$.
The general boltzmann machine defines $p(v,h;\theta) $ through a set of weighted, symmetric connextions between all visible and hidden units.

The restricted Boltzmann machine is an energy-based model with the joint probability distribution speciﬁed by its energy function:
\begin{equation}
P(v,h) = \frac{1}{Z} e^{-E(v,h)}
\end{equation}

The Energy function for RBM is defined as:
\begin{equation}
E(v,h) = -b^{T} v - c^{T} h - v^{T} W h 
\end{equation}

and Z is the partition function which is a normalizing constant given as:
\begin{equation}
Z = \Sigma_{v} \Sigma_{h} e^{-E(v,h)}
\end{equation}


In the case of restricted Boltzmann machines, Long and Servedio (2010) formally proved that the partition function Z is intractable. The intractable partition function Z implies that the normalized joint probability distribution P (v) is also intractable to evaluate. Though P(v) is intractable, the bipartite graph structure of the RBM has the special property that visible and hidden units are conditionally independent given one-another. Therefore:
\begin{equation}
P(h|v) = \frac{P(h,v)}{P(v)}
       = \frac{1}{P(v)} \frac{1}{Z} exp\{b^{T} v + c^{T} h + v^{T} W h\}
\end{equation}

\begin{equation}
    = \frac{1}{Z'} exp\{c^T h + v^{T} W h\}
\end{equation}

\begin{equation}
    = \frac{1}{Z'} exp\{\Sigma_j c_j h_j + \Sigma_j v^T W_j h_j \}
\end{equation}

\begin{equation}
    = \frac{1}{Z'} \Pi_j exp \{ c_j h_j + v^T W_j h_j\}
\end{equation}

\begin{equation}
P(h_j = 1,v) = \frac{\hat{P}(h_j = 1,v)}{\hat{P}(h_j = 0,v) + \hat{P}(h_j = 1,v)}
             = \frac{exp\{c_j + v^T W_j \}}{exp\{0\} + exp\{c_j + v^T W_j \}}
\end{equation}

Therefore:
\begin{equation}
P(h_j = 1|v) = \sigma(c_j + v^T W_j )
\end{equation}


Similary from Eq 8 we can say that:
\begin{equation}
P(v|h) = \frac{1}{Z'} \Pi_k exp \{b_k + h^T W_k \}
\end{equation}
Therefore:
\begin{equation}
P(v_k = 1|h) = \sigma(b_k + h^T W_k)
\end{equation}

In [170]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from mlxtend.data import loadlocal_mnist
from tensorflow.examples.tutorials.mnist import input_data


SyntaxError: from __future__ imports must occur at the beginning of the file (<ipython-input-170-2a678245a9fb>, line 10)

In [16]:
X_train, y_train= loadlocal_mnist(images_path='train-images-idx3-ubyte',
                                  labels_path='train-labels-idx1-ubyte')
X_test, y_test=loadlocal_mnist(images_path='t10k-images-idx3-ubyte',
                                labels_path='t10k-labels-idx1-ubyte')

In [143]:
MNIST = input_data.read_data_sets('../MNIST_data', one_hot=True)
X_dim = MNIST.train.images.shape[1]
y_dim = MNIST.train.labels.shape[1]

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [73]:
# Removing 20% pixels
import cv2

newX,newY = MNIST.test.images.shape[1], MNIST.test.images.shape[0]*0.8
newX1,newY1 = MNIST.test.images.shape[1], MNIST.test.images.shape[0]*0.5
newX2,newY2 = MNIST.test.images.shape[1], MNIST.test.images.shape[0]*0.2

# for i in MNIST20.test.images:
    


# #Removing 50% of pixels
# MNIST50= cv2.resize(MNIST.test.images,(int(newX1),int(newY1)))

# # #Removing 80% of pixels
# MNIST80=cv2.resize(MNIST.test.images,(int(newX2),int(newY2)))


AttributeError: can't set attribute

In [144]:
mb_size=16
h_dim= 500

In [145]:
W = np.random.randn(X_dim, h_dim) * 0.001
a = np.random.randn(h_dim) * 0.001
b = np.random.randn(X_dim) * 0.001

In [146]:
def sigm(x):
    return 1/(1+np.exp(-x))

def infer(x):
    return sigm(x @ W)

def generate(H):
    return sigm(H @ W.T)

alpha=0.1
K = 10

for t in range(1, 1001):
    X_mb = (MNIST.train.next_batch(mb_size)[0] > 0.5).astype(np.float)
    g = 0
    g_a = 0
    g_b = 0

    for v in X_mb:
        
        h = infer(v)

        v_prime = np.copy(v)

        for k in range(K):
            h_prime = np.random.binomial(n=1, p=infer(v_prime))
            v_prime = np.random.binomial(n=1, p=generate(h_prime))

        
        h_prime = infer(v_prime)

        
        grad_w = np.outer(v, h) - np.outer(v_prime, h_prime)
        grad_a = h - h_prime
        grad_b = v - v_prime

       
        g += grad_w
        g_a += grad_a
        g_b += grad_b

   
    g *= 1 / mb_size
    g_a *= 1 / mb_size
    g_b *= 1 / mb_size
    W += alpha * g
    a += alpha * g_a
    b += alpha * g_b


In [147]:
def plot(samples, size, name):
    size = int(size)
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(size, size), cmap='Greys_r')

    plt.savefig('out/{}.png'.format(name), bbox_inches='tight')
    plt.close(fig)

In [148]:
def testModel():
    X = (MNIST.test.next_batch(mb_size)[0] > 0.5).astype(np.float)

    H = np.random.binomial(n=1, p=infer(X))
#     plot(H, np.sqrt(h_dim), 'H')

    X_recon = (generate(H) > 0.5).astype(np.float)
    plot(X_recon, np.sqrt(X_dim), 'V')
    
    

In [149]:
type(MNIST.test)

tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet

In [150]:
testModel()

## 20 Hidden nodes

<img src="out/20Nodes.png"></img>

## 100 Hidden nodes

<img src="out/100Nodes.png"></img>

## 500 Hidden Nodes

<img src="out/500Nodes.png"></img>

## 2. VAE

#### Derive the variational autoencoder algorithm that you will implement, and explain your derivation. Implement the training and inference algorithms for VAE.  Train VAE with 2, 8 and 16 code units to encode MNIST images using the training data set. The neural network will be 784 input -> 256 hidden -> 2/8/16 code -> 256 hidden -> 784 output. Then  use the 2 code -> 256 hidden -> 784 output part of the trained network with 2 code units to generate images by varying each code unit from -3 to 3. You are free to choose the other parameters.

VAE (Variational autoencoder) is rooted in bayesian inference; it wants to model the underlying probability distribution of data so that it can sample new data points from that distribution. VAE uses latent variable, hence it is an expressive model.

NOTATIONS:

- $ X $ Data input for the model <br>
- $ z $ Latent variable <br>
- $ P(X) $ Probability distribution of the data <br>
- $ P(z) $ Probability distribution of latent variable <br>
- $ P(X|z) $ Probability distribution of data given latent variable

Our main goal is to model the data, hence we need to find $P(X)$. THis can be done as follows:

$ P(X) = \int{P(X|z) P(z) dz}$

that is we marginalize the joint probability from the latent variable.

Now we need $P(X,z)$, the idea is to infer $ P(z) $ using $ P(z|X) $. For this we use a method called Variational Inference. The main idea of Variational Inference is to suggest the inference by thinking it as an optimization problem. This can be done by modeling the true distribution $P(z|X)$ using distribution such as Gaussian, and minimize the distribution using KL divergence, which tells us difference between P and Q.

Now we have to infer $ P(z|X)$ using $Q(z|X)$

We want to infer $P(z|X)$ using $Q(z|X)$, the KL divergence then formulated as follows:
\begin{equation}
D_{KL}[Q(z|X)∥P(z|X)] = \Sigma_z Q(z|X) log \frac{Q(z|X)}{P(z|X)}
                      = E[log Q(z|X) - log P(z|X)]
\end{equation}

By Baye's rule:
\begin{equation}
D_{KL}[Q(z|X)∥P(z|X)] = E[log Q(z|X) - log \frac{P(X|z) P(z)}{P(X)}]
= E[log Q(z|X) − log P(X|z) − log P(z) + log P(X)]
\end{equation}

The expectation is over z and P(X) doesn’t depend on z, so we could move it outside of the expectation:
\begin{equation}
D_{KL}[Q(z|X)∥P(z|X)] = E[log Q(z|X) − log P(X|z) − log P(z) ] + log P(X)
\end{equation}
\begin{equation}
D_{KL}[Q(z|X)∥P(z|X)] - log P(X)  = E[log Q(z|X) − log P(X|z) − log P(z)]
\end{equation}

The right hand side of Eq 15 can be expressed as another KL divergence (Rearranging signs):
\begin{equation}
log P(X) - D_{KL}[Q(z|X)∥P(z|X)] = E[log P(X|z) - (log Q(z|X) − log P(z))]
= E[log P(X|z)] - E[log Q(z|X) - log P(z)]
\end{equation}
This is the VAE objective function:
\begin{equation}
log P(X) - D_{KL}[Q(z|X)∥P(z|X)] = E[log P(X|z)] - D_{KL}[Q(z|X)∥P(z)]
\end{equation}



Importing libraries to implement VAE

- We are using Vanilla VAE, as it is the simplest form of VAE, as it does not use Neural network for implementing the layers.

In [121]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from tensorflow.examples.tutorials.mnist import input_data

- Initializing data and the dimensions for the variables.

In [122]:
mnist = input_data.read_data_sets('../MNIST_data', one_hot=True)
mb_size = 64
z_dim = 16
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 256
c = 0
lr = 1e-3

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


- Function to plot images and to initilize data and xavier function

In [123]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='magma')

    return fig


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [124]:
X = tf.placeholder(tf.float32, shape=[None, X_dim])
z = tf.placeholder(tf.float32, shape=[None, z_dim])

Q_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

Q_W2_mu = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_mu = tf.Variable(tf.zeros(shape=[z_dim]))

Q_W2_sigma = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_sigma = tf.Variable(tf.zeros(shape=[z_dim]))


def Q(X):
    h = tf.nn.relu(tf.matmul(X, Q_W1) + Q_b1)
    z_mu = tf.matmul(h, Q_W2_mu) + Q_b2_mu
    z_logvar = tf.matmul(h, Q_W2_sigma) + Q_b2_sigma
    return z_mu, z_logvar


def sample_z(mu, log_var):
    eps = tf.random_normal(shape=tf.shape(mu))
    return mu + tf.exp(log_var / 2) * eps


In [125]:
P_W1 = tf.Variable(xavier_init([z_dim, h_dim]))
P_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

P_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
P_b2 = tf.Variable(tf.zeros(shape=[X_dim]))


def P(z):
    h = tf.nn.relu(tf.matmul(z, P_W1) + P_b1)
    logits = tf.matmul(h, P_W2) + P_b2
    prob = tf.nn.sigmoid(logits)
    return prob, logits

In [126]:
z_mu, z_logvar = Q(X)
z_sample = sample_z(z_mu, z_logvar)
_, logits = P(z_sample)
X_samples, _ = P(z)

recon_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=X), 1)
kl_loss = 0.5 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)
vae_loss = tf.reduce_mean(recon_loss + kl_loss)
solver = tf.train.AdamOptimizer().minimize(vae_loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

for it in range(10000):
    X_mb, _ = mnist.train.next_batch(mb_size)

    _, loss = sess.run([solver, vae_loss], feed_dict={X: X_mb})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('Loss: {:.4}'. format(loss))
        print()

        samples = sess.run(X_samples, feed_dict={z: np.random.randn(16, z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

Iter: 0
Loss: 747.8

Iter: 1000
Loss: 118.5

Iter: 2000
Loss: 115.6

Iter: 3000
Loss: 112.8

Iter: 4000
Loss: 111.2

Iter: 5000
Loss: 112.2

Iter: 6000
Loss: 103.6

Iter: 7000
Loss: 109.6

Iter: 8000
Loss: 114.3

Iter: 9000
Loss: 114.0



## 2 Code Units for Latent space and 256 Hidden nodes

<img src="out/2code.png"></img>

## 8 Code Units for Latent space and 256 Hidden nodes

<img src="out/8code.png"></img>

## 16 Code Units for Latent space and 256 Hidden nodes

<img src="out/16code.png"></img>

# 3. 

### 3. (Optional) Implement VAE to generate MNIST images, where you use convolutional neural network from encoder and deconvolutional neural network for decoder. This one should be super-easy (because there are many tutorials in the Internet and you have already solved problem 2) for you to catch up with the total score.




In [202]:
from keras.datasets import mnist
import keras.initializers
from keras import backend as K

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K
from keras import objectives
from keras.utils import np_utils

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [203]:
image_size = x_train.shape[1]
original_dim = image_size * image_size

In [204]:
intermediate_dim = 512
batch_size = 128
latent_dim = 2
epochs = 50

In [205]:
x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_sigma = Dense(latent_dim)(h)

In [206]:
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim))
    return z_mean + K.exp(z_log_sigma) * epsilon

`
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

In [207]:
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

In [208]:

vae = Model(x, x_decoded_mean)

encoder = Model(x, z_mean)

decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

In [209]:
def vae_loss(x, x_decoded_mean):
    xent_loss = objectives.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return xent_loss + kl_loss


vae.compile(optimizer='rmsprop', loss=vae_loss)

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test,10)
vae.fit(x_train,x_train, shuffle=True,epochs=epochs,batch_size=batch_size,
        validation_data=(x_test,x_test))
       

## Digits generated by VAE

<img src="VAE.png"></img>

### 4. (Optional) Derive and explain the GAN algorithm. Implement GAN and train it from MNIST training data set to generate digits. Show images generated from GAN.



In [ ]:
X = tf.placeholder(tf.float32, shape=[None, 784])

D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


Z = tf.placeholder(tf.float32, shape=[None, 100])

G_W1 = tf.Variable(xavier_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]


def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='magma')

    return fig


G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)


D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

mb_size = 128
Z_dim = 100

mnist = input_data.read_data_sets('../MNIST_data', one_hot=True)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

for it in range(50000):
    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))

## Image generated by GAN

<img src="GAN.png"></img>

# REFERENCES

1. https://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/

2. http://deeplearning.net/tutorial/rbm.html

3. https://blog.keras.io/building-autoencoders-in-keras.html

4. Textbook , slides

5. https://skymind.ai/wiki/restricted-boltzmann-machine

6. https://rubikscode.net/2018/10/01/introduction-to-restricted-boltzmann-machines/

7. https://rubikscode.net/2018/10/22/implementing-restricted-boltzmann-machine-with-python-and-tensorflow/

8. WISEODD

9. https://medium.com/@manivannan_data/resize-image-using-opencv-python-d2cdbbc480f0

10. https://github.com/wiseodd/generative-models